In [6]:
pip install beautifulsoup4

  Using cached beautifulsoup4-4.13.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.13.3-py3-none-any.whl (186 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from bs4 import BeautifulSoup
import json
import re

# json 파일 확인

In [9]:
with open("../../Database/input/구강_구개열_crop_parsing.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 최상위 키 확인
print("🔑 최상위 keys:", data.keys())

# content 내부 키
if 'content' in data:
    print("🔑 content keys:", data['content'].keys())

# content 안에 'elements'가 있다면, 첫 번째 요소 구조 확인
if 'elements' in data['content']:
    print("📌 elements[0] 예시:")
    print(data['content']['elements'][0].keys())

for k, v in data.items():
    print(f"{k}: {type(v)}")
    if isinstance(v, dict):
        for kk in v.keys():
            print(f"  └─ {kk}: {type(v[kk])}")

🔑 최상위 keys: dict_keys(['api', 'content', 'elements', 'merged_elements', 'model', 'ocr', 'usage'])
🔑 content keys: dict_keys(['html', 'markdown', 'text'])
api: <class 'str'>
content: <class 'dict'>
  └─ html: <class 'str'>
  └─ markdown: <class 'str'>
  └─ text: <class 'str'>
elements: <class 'list'>
merged_elements: <class 'list'>
model: <class 'str'>
ocr: <class 'bool'>
usage: <class 'dict'>
  └─ pages: <class 'int'>


# 병명 추출

In [10]:
html = data['content']['html']
soup = BeautifulSoup(html, 'html.parser')

# 병명 추출 (h1 없을 때 대비)
disease_tag = soup.find("h1")

if disease_tag is None:
    print("[⚠️경고] h1 태그를 찾을 수 없습니다.")
    
    # h1이 없으면 font-size:22px인 첫 <p>나 <header>를 찾아서 병명 추출
    for tag in soup.find_all(["p", "header"]):
        style = tag.get("style", "")
        if "font-size:22px" in style:
            disease_tag = tag
            print("[⚠️] fontsize 22 태그로 추출했습니다.")

            break

# 병명 최종 추출
if disease_tag:
  disease_name = disease_tag.get_text(strip=True)
else:
  disease_name = "Unknown Disease"
  print("[⚠️경고] 병명 추출 실패: h1 또는 font-size:22px 태그를 찾을 수 없습니다.")

# 3. 그래도 없으면 '• 콘텐츠명 :' 포함된 텍스트에서 추출
if not disease_name:
    for tag in soup.find_all("p"):
        text = tag.get_text(strip=True)
        if "• 콘텐츠명" in text:
            # "• 콘텐츠명 : 구개열" 형식에서 오른쪽 값만 추출
            match = re.search(r"콘텐츠명\s*:\s*(.+)", text)
            if match:
                disease_name = match.group(1).strip()
                break
            
print(f'✅병명: {disease_name}')


✅병명: 구개열


# 내용 추출

In [11]:
chunks = []
current_section = None
current_text = []

first_page_font_threshold = 22
other_page_font_threshold = 18

for elem in data['elements']:
    html = elem.get("content", {}).get("html", "")
    text = BeautifulSoup(html, "html.parser").get_text(strip=True)
    page = elem.get("page", 1)
    category = elem.get("category", "")
    style = ""
    
    # style 정보 추출
    soup = BeautifulSoup(html, "html.parser")
    tag = soup.find()
    if tag:
        style = tag.get("style", "")

    # font-size 추출
    font_size = 0
    if "font-size:" in style:
        try:
            font_size = int(style.split("font-size:")[1].replace("px", "").strip())
        except:
            pass

    # 페이지별 기준 적용
    threshold = first_page_font_threshold if page == 1 else other_page_font_threshold
    is_heading = (category.startswith("heading") or font_size >= threshold)

    if not text:
        continue

    if is_heading:
        if current_section and current_text:
            chunks.append({
                "disease": disease_name,
                "section": current_section,
                "content": "\n".join(current_text),
                "page": page
            })
            current_text = []

        current_section = text
    else:
        current_text.append(text)

# 마지막 섹션 저장
if current_section and current_text:
    chunks.append({
        "disease": disease_name,
        "section": current_section,
        "content": "\n".join(current_text),
        "page": page
    })

## json으로 저장

In [13]:
print("done!")

# 결과를 JSON 파일로 저장 (가독성 있게)
with open("../../Database/output/processed/구개열_crop_test.json", "w", encoding="utf-8") as f_json:
    json.dump(chunks, f_json, ensure_ascii=False, indent=2)

# 또는 텍스트 파일로 보기 좋게 저장
with open("../../Database/output/processed/구개열_crop_test.txt", "w", encoding="utf-8") as f_txt:
    for chunk in chunks:
        f_txt.write(f"[{chunk['section']}]\n")
        f_txt.write(chunk["content"] + "\n\n")


done!
